In [2]:
from dotenv import load_dotenv
import os 

from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document




c:\Users\vyshn\Desktop\JobFinder\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv
groq_api = os.getenv("GROQ_API_KEY")
print(groq_api)


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


In [4]:
base_dir = os.getcwd()
products_file_path = os.path.join(base_dir, "ProductBacklog_v1.txt")

with open(products_file_path, "r" , encoding="utf-8") as file:
    products = file.read()

# print(products)
# print(len(products))


In [5]:
llm = ChatGroq(api_key=groq_api, model="openai/gpt-oss-120b", temperature=0)
SYSTEM_PROMPT = f"""
you are a job finder assistant , your task is to be a a friendly support bot for the Job Finder website.
Your job:
- Answer questions about how Job Finder works.
- Only use the information given below about the product.
- If the user asks about something we don't have yet, say it's not available yet.

Product information:
{products}


"""

In [10]:
import sqlite3
from datetime import datetime

DB_PATH = "chat_history.db"

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS messages (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id TEXT NOT NULL,
            role TEXT NOT NULL,
            content TEXT NOT NULL,
            created_at TEXT NOT NULL
        )
    """)
    conn.commit()
    conn.close()

def save_message(user_id: str, role: str, content: str):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute(
        "INSERT INTO messages (user_id, role, content, created_at) VALUES (?, ?, ?, ?)",
        (user_id, role, content, datetime.utcnow().isoformat())
    )
    conn.commit()
    conn.close()

def load_history(user_id: str, limit: int = 20):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute(
        """
        SELECT role, content
        FROM messages
        WHERE user_id = ?
        ORDER BY id DESC
        LIMIT ?
        """,
        (user_id, limit)
    )
    rows = cur.fetchall()
    conn.close()
    
    rows.reverse()  # oldest first
    return rows

init_db()
print("SQLite conversation DB ready.")


SQLite conversation DB ready.


In [8]:
# basic chatbot interaction
# messages = [
#     {"role": "system" , "content": SYSTEM_PROMPT},
#     {"role": "user" , "content": "what are the kwy features of Job Finder?"},
# ]

# result = llm.invoke(messages)
# print(result)

In [13]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

# Load your file
with open("ProductBacklog_v1.txt", "r", encoding="utf-8") as f:
    PRODUCT_INFO = f.read()

emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

docs = [Document(page_content=PRODUCT_INFO.strip())]

vectorstore = Chroma.from_documents(
    docs,
    embedding=emb,
    collection_name="jobfinder_kb",
    persist_directory="./chroma_store"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

print("Chroma knowledge DB ready.")


C:\Users\vyshn\AppData\Local\Temp\ipykernel_2528\4047929860.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Chroma knowledge DB ready.


In [22]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

SYSTEM_PROMPT = """
You are Job Finder Assistant...
"""

def ask_bot(user_id: str, user_question: str) -> str:

    # 1) Load conversation history from SQLite
    history_rows = load_history(user_id, limit=20)

    messages = [SystemMessage(content=SYSTEM_PROMPT)]

    for role, content in history_rows:
        if role == "user":
            messages.append(HumanMessage(content=content))
        else:
            messages.append(AIMessage(content=content))

    # 2) Get relevant knowledge from Chroma
    relevant_docs  = retriever.invoke(user_question)
    context_text = "\n\n".join(d.page_content for d in relevant_docs)

    # 3) Add new user message including context
    combined_input = f"Context:\n{context_text}\n\nUser question: {user_question}"
    messages.append(HumanMessage(content=combined_input))

    # 4) Call LLM
    result = llm.invoke(messages)
    reply_text = result.content

    # 5) Save history in SQLite
    save_message(user_id, "user", user_question)
    save_message(user_id, "assistant", reply_text)

    return reply_text


In [18]:
print(ask_bot("user1", "What is Job Finder?"))
print(ask_bot("user1", "Do you support other countries?"))


**Job Finder** is a free, AI‑powered job‑search platform for the United States that pulls fresh, real job postings every 30 minutes, removes duplicates and scams, and matches openings to your skills, experience, and preferences. It lets you filter by skills, location, job type (contract, full‑time, remote, etc.), upload a PDF/DOCX résumé for better matching, and track your applications—all in a clean, clutter‑free interface.
**No – at the moment Job Finder only supports job listings in the United States.**  

We focus on U.S.–based roles to ensure the data is fresh, vetted and compliant with local hiring practices. Expanding to other countries is on our roadmap, but there’s no official timeline yet. If you’re looking for opportunities outside the U.S., you’ll need to use a different job‑search tool for now.


In [19]:
print(ask_bot("user2", "why is Job Finder?"))
print(ask_bot("user2", "Do you show h1b jobs?"))

**Why Job Finder exists – its purpose and the problem it solves**

Job Finder was created to give job‑seekers in the United States a **fast, clean, and trustworthy way to find real work**—whether they’re looking for contract, full‑time, part‑time, or remote positions.  

| The pain points people face today | How Job Finder solves them |
|-----------------------------------|----------------------------|
| **Stale or duplicate listings** – many job boards refresh only once a day or show the same posting dozens of times. | **Real‑time updates** every 30 minutes and automatic duplicate removal keep the feed fresh and concise. |
| **Fake or scam postings** – endless spam, “too‑good‑to‑be‑true” offers, and non‑US jobs clutter the search. | **AI‑driven filtering** strips out scam‑like and non‑US listings, so you only see legitimate U.S. opportunities. |
| **Time‑consuming manual filtering** – you have to sift by location, experience, visa status, etc., on every site. | **Smart job filtering**

In [21]:
print(ask_bot("user1", "what all did i ask you before?"))

Here’s a quick recap of the questions you’ve asked so far in this conversation:

1. **“What is Job Finder?”** – You wanted a description of the platform.  
2. **“Do you support other countries?”** – You asked whether Job Finder works outside the United States.  
3. **“What did I ask you before?”** – You asked for a reminder of your previous questions (which is what you’re reading now).
